<a href="https://colab.research.google.com/github/mag381/mukonoso/blob/main/YouTube%E5%8B%95%E7%94%BB%E3%83%95%E3%83%AC%E3%83%BC%E3%83%A0%E6%8A%BD%E5%87%BA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

YouTube動画フレーム抽出ツール </p>
https://qiita.com/ussu_ussu_ussu/items/ff4e7f98947eae84983e

In [1]:
# @title セットアップ { display-mode: "form" }
!pip install yt-dlp opencv-python-headless

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.2/172.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 47.8 MB/s eta 0:00:00


--- 設定 ---

In [2]:
#動画のタイトルなど自由に
youtube_url = "https://youtu.be/.....output_image_path = "output_frames.png"
temp_video_path = "temp_video.mp4"
frame_interval = 300  # 何フレームごとに抽出するか (1秒あたり約30フレームなら、30で約1秒ごと)

必要なライブラリのインポート

In [3]:
import yt_dlp
import cv2
import numpy as np
from PIL import Image
import os

YouTube動画のダウンロード

In [4]:
# --- yt-dlp のオプション設定 ---
ydl_opts = {
    'format': 'bestvideo[ext=mp4]/best[ext=mp4]/best', # 音声なしでOK
    'outtmpl': temp_video_path,
    'noplaylist': True,
}

# --- YouTube動画のダウンロード ---
try:
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([youtube_url])
except yt_dlp.utils.DownloadError as e:
    print(f"ダウンロードエラー: {e}")
    exit()

[youtube] Extracting URL: https://youtu.be/2PBLuquEjBc
[youtube] 2PBLuquEjBc: Downloading webpage
[youtube] 2PBLuquEjBc: Downloading tv client config
[youtube] 2PBLuquEjBc: Downloading player 22f02d3d-main
[youtube] 2PBLuquEjBc: Downloading tv player API JSON
[youtube] 2PBLuquEjBc: Downloading ios player API JSON
[youtube] 2PBLuquEjBc: Downloading m3u8 information
[info] 2PBLuquEjBc: Downloading 1 format(s): 137
[download] Destination: temp_video.mp4
[download] 100% of   34.90MiB in 00:00:12 at 2.75MiB/s   


フレーム抽出プロセス

In [5]:
# --- 動画の読み込み ---
cap = cv2.VideoCapture(temp_video_path)
if not cap.isOpened():
    print("動画ファイルを開けませんでした")
    exit()

# --- フレームの抽出と結合 ---
frame_count = 0
all_frames = []  # 抽出したフレームを一時的に保存するリスト
while True:
    ret, frame = cap.read()
    if not ret:
        break  # フレームがなくなったら終了
    frame_count += 1
    # 指定した間隔でフレームを抽出
    if frame_count % frame_interval == 0:
        all_frames.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)) # OpenCV -> PIL互換形式

cap.release()
os.remove(temp_video_path)  # 一時動画ファイルは削除

画像結合処理

In [6]:
# --- 画像の結合 (横に並べる) ---
if not all_frames:
    print("フレームが抽出されませんでした")
    exit()

# 最初のフレームから、サイズを取得
frame_height = all_frames[0].shape[0]
frame_width = all_frames[0].shape[1]

# 全体の画像のサイズを計算
total_width = frame_width * len(all_frames)
total_height = frame_height

# 新しい画像を作成
final_image = Image.new("RGB", (total_width, total_height))

# 各フレームを貼り付け
x_offset = 0
for frame_array in all_frames:
  frame_img = Image.fromarray(frame_array)
  final_image.paste(frame_img, (x_offset, 0))
  x_offset += frame_width

# --- 画像の保存 ---
final_image.save(output_image_path)
print(f"画像を {output_image_path} に保存しました。")

画像を output_frames.png に保存しました。


画像のダウンロード（Colab専用機能）

In [7]:
# --- ダウンロード (Colab用) ---
# @markdown 生成された画像をダウンロードします。
from google.colab import files
files.download(output_image_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>